In [1]:
import boto3
import os
import re
from tqdm import tqdm
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, IntegerType, BooleanType, FloatType
import pyspark.sql.functions as F

In [2]:
os.getcwd()

'/home/jovyan/work/notebooks'

In [3]:
os.chdir('../')

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName('assignment_1') \
        .getOrCreate()

In [5]:
df = spark.read.parquet('data/combined_cleaned_data.parquet')

In [6]:
df.show()

+--------+------------------+----------+------------+------------+---------------+-------------+-----+-------+----------+------------+---------------------+------------+------------+-----------+-------------------+-------------------+
|VendorID|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|taxi_colour|        pickup_date|       dropoff_date|
+--------+------------------+----------+------------+------------+---------------+-------------+-----+-------+----------+------------+---------------------+------------+------------+-----------+-------------------+-------------------+
|       2|             false|         1|          66|          33|              5|         0.51|  0.5|    0.5|       0.7|         0.0|                  0.3|           6|           1|      green|2018-06-01 00:33:55|2018-06-01 00:36:13|
|       2|             false|         1|          25|       

# 2. Transformations

In [7]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- taxi_colour: string (nullable = true)
 |-- pickup_date: timestamp (nullable = true)
 |-- dropoff_date: timestamp (nullable = true)



In [8]:
df.createOrReplaceTempView("combined_data")

## 4. SQL Queries


### 4a. For each year and month:
***i. What was the total number of trips***

In [9]:
query = """SELECT COUNT(*) as trips, MONTH(pickup_date) as month, YEAR(pickup_date) as year
FROM combined_data
GROUP BY MONTH(pickup_date), YEAR(pickup_date)
ORDER BY YEAR(pickup_date) desc,  MONTH(pickup_date) desc"""

In [10]:
spark.sql(query).show(24)

+--------+-----+----+
|   trips|month|year|
+--------+-----+----+
| 8857779|   12|2018|
| 8802332|   11|2018|
| 9531620|   10|2018|
| 8706559|    9|2018|
| 8515365|    8|2018|
| 8534027|    7|2018|
| 9453060|    6|2018|
|10021377|    5|2018|
|10105415|    4|2018|
|10266415|    3|2018|
| 9262273|    2|2018|
| 9553631|    1|2018|
|10414465|   12|2017|
|10158994|   11|2017|
|10694462|   10|2017|
| 9827956|    9|2017|
| 9289554|    8|2017|
| 9503269|    7|2017|
|10633460|    6|2017|
|11161587|    5|2017|
|11127959|    4|2017|
|11453266|    3|2017|
|10192088|    2|2017|
|10780387|    1|2017|
+--------+-----+----+



***ii. Which weekday had the most trips?***
For ease in the SQL statement, I have created columns for the relevant parts of the datetime. I could have used these functions within the SQL statement itself.

In [7]:
from pyspark.sql.functions import year, month, dayofweek
import pyspark.sql.functions as F

df = df.withColumn('week_day', dayofweek(F.col('pickup_date')))
df = df.withColumn('month', month(F.col('pickup_date')))
df = df.withColumn('year', year(F.col('pickup_date')))

In [30]:
df.createOrReplaceTempView("combined_data")

In [31]:
query = """select b1.trips,b2.week_day, b1.month,b1.year from
(select max(b.trips) as trips, b.month, b.year  from 
    (SELECT COUNT(*) as trips, week_day,month, year
FROM combined_data
GROUP BY week_day,month, year) as b
Group by b.month, b.year) as b1
inner join (SELECT COUNT(*) as trips, week_day,month, year
FROM combined_data
GROUP BY week_day,month, year) as b2 
on (b1.trips, b1.month,b1.year)=(b2.trips, b2.month,b2.year)
ORDER BY b2.year desc,  b2.month desc"""

In [32]:
spark.sql(query).show(24)

+-------+--------+-----+----+
|  trips|week_day|month|year|
+-------+--------+-----+----+
|1508664|       7|   12|2018|
|1524979|       6|   11|2018|
|1577684|       4|   10|2018|
|1472752|       7|    9|2018|
|1488433|       4|    8|2018|
|1456883|       3|    7|2018|
|1645423|       6|    6|2018|
|1745106|       5|    5|2018|
|1523951|       2|    4|2018|
|1811863|       6|    3|2018|
|1464827|       6|    2|2018|
|1628165|       4|    1|2018|
|1831381|       6|   12|2017|
|1743664|       4|   11|2017|
|1676870|       3|   10|2017|
|1725010|       6|    9|2017|
|1606769|       5|    8|2017|
|1529962|       7|    7|2017|
|1855817|       5|    6|2017|
|1861369|       4|    5|2017|
|1969259|       7|    4|2017|
|2034770|       6|    3|2017|
|1616477|       7|    2|2017|
|1701968|       3|    1|2017|
+-------+--------+-----+----+



***iii. Which hour of the day had the most trips?***

In [8]:
from pyspark.sql.functions import year, month, dayofweek, hour
import pyspark.sql.functions as F

df = df.withColumn('hour', hour(F.col('pickup_date')))

In [34]:
df.createOrReplaceTempView("combined_data")

In [35]:
query = """select b1.trips,b2.hour, b1.month,b1.year from
(select max(b.trips) as trips, b.month, b.year  from 
    (SELECT COUNT(*) as trips, hour,month, year
FROM combined_data
GROUP BY hour,month, year) as b
Group by b.month, b.year) as b1
inner join (SELECT COUNT(*) as trips,hour,month, year
FROM combined_data
GROUP BY hour,month, year) as b2 
on (b1.trips, b1.month,b1.year)=(b2.trips, b2.month,b2.year)
ORDER BY b2.year desc,  b2.month desc"""

In [36]:
spark.sql(query).show(24)

+------+----+-----+----+
| trips|hour|month|year|
+------+----+-----+----+
|551515|  18|   12|2018|
|555520|  18|   11|2018|
|612721|  18|   10|2018|
|557282|  18|    9|2018|
|553496|  18|    8|2018|
|547694|  18|    7|2018|
|591743|  18|    6|2018|
|636948|  18|    5|2018|
|661268|  18|    4|2018|
|667507|  18|    3|2018|
|614335|  18|    2|2018|
|632957|  18|    1|2018|
|647202|  18|   12|2017|
|650704|  18|   11|2017|
|684103|  18|   10|2017|
|622207|  19|    9|2017|
|588260|  18|    8|2017|
|589087|  18|    7|2017|
|656818|  18|    6|2017|
|690046|  18|    5|2017|
|697420|  18|    4|2017|
|735218|  19|    3|2017|
|673287|  18|    2|2017|
|692726|  18|    1|2017|
+------+----+-----+----+



***iv. What was the average number of passengers?***

In [46]:
df.createOrReplaceTempView("combined_data")

In [47]:
query = """SELECT AVG(passenger_count), month,  year
FROM combined_data
GROUP BY  month, year
ORDER BY year desc,  month desc"""

In [48]:
spark.sql(query).show(24)

+--------------------+-----+----+
|avg(passenger_count)|month|year|
+--------------------+-----+----+
|  1.5749735910096652|   12|2018|
|  1.5589607390405178|   11|2018|
|  1.5523811272375525|   10|2018|
|  1.5698228197844866|    9|2018|
|  1.5822324703638657|    8|2018|
|  1.5859522122439969|    7|2018|
|  1.5787771367155186|    6|2018|
|   1.577745653117331|    5|2018|
|  1.5817253423041013|    4|2018|
|  1.5817686115357699|    3|2018|
|  1.5763987954144734|    2|2018|
|  1.5865468322986307|    1|2018|
|   1.609397794317807|   12|2017|
|  1.5904028489435076|   11|2017|
|  1.5955872301009626|   10|2017|
|  1.6032556515312035|    9|2017|
|  1.6093790939801846|    8|2017|
|  1.6151521123941668|    7|2017|
|  1.5992691936585082|    6|2017|
|  1.5952257506033864|    5|2017|
|   1.601502935084502|    4|2017|
|  1.5923528712246795|    3|2017|
|  1.5986640813933317|    2|2017|
|  1.6031179585667936|    1|2017|
+--------------------+-----+----+



***v. What was the average amount paid per trip (total_amount)?***

In [49]:
df.createOrReplaceTempView("combined_data")

In [50]:
query = """SELECT AVG(total_amount), month,  year
FROM combined_data
GROUP BY  month, year
ORDER BY year desc,  month desc"""

In [51]:
spark.sql(query).show(24)

+---------------------------------+-----+----+
|avg(CAST(total_amount AS DOUBLE))|month|year|
+---------------------------------+-----+----+
|                 16.5143992903445|   12|2018|
|                 16.7899788057978|   11|2018|
|               16.947767601974277|   10|2018|
|               16.808269310715303|    9|2018|
|                16.57538040690682|    8|2018|
|                16.54223904844393|    7|2018|
|               16.628314385004405|    6|2018|
|               16.755300685929303|    5|2018|
|                16.27453824528984|    4|2018|
|               15.879032786926034|    3|2018|
|               15.366286035642922|    2|2018|
|               15.367568301570396|    1|2018|
|                16.01089971694115|   12|2017|
|               16.305887479662697|   11|2017|
|               16.554977008867855|   10|2017|
|               16.535440411042156|    9|2017|
|               16.289620089609635|    8|2017|
|               16.190408010335542|    7|2017|
|            

***vi. What was the average amount paid per passenger (total_amount)?***

In [55]:
df.createOrReplaceTempView("combined_data")

In [56]:
query = """SELECT AVG(total_amount/passenger_count) as cost_per_passenger, month,  year
FROM combined_data
GROUP BY  month, year
ORDER BY year desc,  month desc"""

In [57]:
spark.sql(query).show(24)

+------------------+-----+----+
|cost_per_passenger|month|year|
+------------------+-----+----+
|13.549758973470507|   12|2018|
|13.920740461645034|   11|2018|
|14.085970087706691|   10|2018|
|13.913103221285503|    9|2018|
|13.676620190797392|    8|2018|
| 13.63917960186089|    7|2018|
|13.734457836029895|    6|2018|
|13.857840071732484|    5|2018|
|13.420629305781736|    4|2018|
|13.118663687856754|    3|2018|
|12.744973055655787|    2|2018|
|12.705123826288716|    1|2018|
|13.084823013559964|   12|2017|
|13.452684448129276|   11|2017|
|13.656854595631154|   10|2017|
|13.610389402410114|    9|2017|
|13.378996400182738|    8|2017|
|13.266759951874388|    7|2017|
|13.585708949705127|    6|2017|
| 13.63220577948025|    5|2017|
|13.296277392036433|    4|2017|
|13.226725031818637|    3|2017|
|12.746678349579957|    2|2017|
|12.648085591722742|    1|2017|
+------------------+-----+----+



### b. For each taxi colour (yellow and green)

***i. What was the average, median, minimum and maximum trip duration in seconds?***

In [6]:
df = df.withColumn('trip_duration',F.col("dropoff_date").cast("long") - F.col('pickup_date').cast("long"))

In [9]:
df.createOrReplaceTempView("combined_data")

In [10]:
query = """SELECT AVG(trip_duration) as average, percentile_approx(trip_duration, 0.5) as median, MIN(trip_duration) as minimum,MAX(trip_duration) as maximum , taxi_colour
FROM combined_data
GROUP BY taxi_colour
"""

In [11]:
spark.sql(query).show()

+------------------+------+-----------+--------+-----------+
|           average|median|    minimum| maximum|taxi_colour|
+------------------+------+-----------+--------+-----------+
|1263.6345686688685|   625|   -1437165|  202989|      green|
| 995.1493983581707|   669|-2911502804|45466304|     yellow|
+------------------+------+-----------+--------+-----------+



While the average and median results appear reasonable, the minimum and maximum values appear to be flawed. Duration cannot be negative, and the maximum values are approximately 2 days and 526 days respectively. It is possible a taxi could have been rented for 2 days, however 526 days is highly unlikely. 

***ii. What was the average, median, minimum and maximum trip distance in km?***
The data dictionary states that the unit of measurement is a mile for trip distance. Value for conversion was retrieved from Googles 'unit converter', identified through the search term: "km in miles"

In [7]:
miles_to_km = 1.60934

In [8]:
df = df.withColumn('distance_km', F.col('trip_distance')*miles_to_km)

In [14]:
df.createOrReplaceTempView("combined_data")

In [15]:
query = """SELECT AVG(distance_km) as average, percentile_approx(distance_km, 0.5) as median, MIN(distance_km) as minimum,MAX(distance_km) as maximum , taxi_colour
FROM combined_data
GROUP BY taxi_colour
"""

In [16]:
spark.sql(query).show()

+-----------------+------------------+-------+------------------+-----------+
|          average|            median|minimum|           maximum|taxi_colour|
+-----------------+------------------+-------+------------------+-----------+
| 4.66496505050157|2.9129053079128266|    0.0|12883.861334091796|      green|
|4.718332105930787| 2.591037423021793|    0.0|  304943.929100625|     yellow|
+-----------------+------------------+-------+------------------+-----------+



***iii. What was the average, median, minimum and maximum speed in km per hour?***

In [9]:
df = df.withColumn('km_per_hour', F.col('distance_km')*3600/(F.col('trip_duration')))

In [19]:
df.createOrReplaceTempView("combined_data")

In [20]:
query = """SELECT AVG(km_per_hour) as average, percentile_approx(km_per_hour, 0.5) as median, MIN(km_per_hour) as minimum,MAX(km_per_hour) as maximum , taxi_colour
FROM combined_data
GROUP BY taxi_colour
"""

In [21]:
spark.sql(query).show()

+------------------+------------------+-------------------+--------------------+-----------+
|           average|            median|            minimum|             maximum|taxi_colour|
+------------------+------------------+-------------------+--------------------+-----------+
| 22.55406684028411|17.743814347767223| -80.12458723404256|  194955.45644036864|      green|
|21.121583492936892|15.872941883537365|-1004.6879889689855|1.6385046936749998E7|     yellow|
+------------------+------------------+-------------------+--------------------+-----------+



### c. What was the percentage of trips where the driver received tips?

In [14]:
df = df.withColumn('tip_received', F.col('tip_amount')>0)

In [24]:
df.createOrReplaceTempView("combined_data")

In [25]:
query = """ SELECT tip_received, count(tip_received) as tip,
       count(tip_received) * 100.0 / (select count(*) from combined_data) as tip_percent
FROM combined_data
group by tip_received"""

In [26]:
spark.sql(query).show()

+------------+---------+-----------------+
|tip_received|      tip|      tip_percent|
+------------+---------+-----------------+
|        true|149053314|62.93224115284405|
|       false| 87793986|37.06775884715595|
+------------+---------+-----------------+



### d. For trips where the driver received tips, What was the percentage where the driver received tips of at least $10.

In [15]:
df = df.withColumn('tip_received_over10', F.col('tip_amount')>=10)

In [33]:
df.createOrReplaceTempView("combined_data")

In [38]:
query = """ SELECT tip_received_over10, count(tip_received_over10) as tip,
       count(tip_received_over10) * 100.0 / (select count(*) from combined_data where tip_received = true) as tip_percent_over10
FROM combined_data
where tip_received = true
group by tip_received_over10"""

In [39]:
spark.sql(query).show()

+-------------------+---------+------------------+
|tip_received_over10|      tip|tip_percent_over10|
+-------------------+---------+------------------+
|               true|  4443013|  2.98082134557572|
|              false|144610301| 97.01917865442428|
+-------------------+---------+------------------+



### e. Classify each trip into bins
***i. Average Speed***

In [10]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[float('-Inf'),0,300,600,1200,1800,float('Inf') ],inputCol="trip_duration", outputCol="trip_duration_bins")
df = bucketizer.setHandleInvalid("keep").transform(df)


In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0:"negative time - Invalid",1.0:"0-5 mins", 2.0: "5-10 mins", 3.0:"10-20 mins", 4.0: "20-30 mins", 5.0: 'at least 30 mins'}
convert_to_text = udf(lambda x: t[x], StringType())
df=df.withColumn("trip_duration_bins_string", convert_to_text("trip_duration_bins"))

In [13]:
df.show(5)

+--------+------------------+----------+------------+------------+---------------+-------------+-----+-------+----------+------------+---------------------+------------+------------+-----------+-------------------+-------------------+-------------+------------------+------------------+------------------+-------------------------+
|VendorID|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|taxi_colour|        pickup_date|       dropoff_date|trip_duration|       distance_km|       km_per_hour|trip_duration_bins|trip_duration_bins_string|
+--------+------------------+----------+------------+------------+---------------+-------------+-----+-------+----------+------------+---------------------+------------+------------+-----------+-------------------+-------------------+-------------+------------------+------------------+------------------+-------------------------+
|   

In [12]:
df.createOrReplaceTempView("combined_data")

In [13]:
query = """SELECT AVG(km_per_hour), AVG(distance_km/total_amount), trip_duration_bins_string
from combined_data
group by trip_duration_bins_string
"""

In [14]:
spark.sql(query).show()

+-------------------+-------------------------------------------------+-------------------------+
|   avg(km_per_hour)|avg((distance_km / CAST(total_amount AS DOUBLE)))|trip_duration_bins_string|
+-------------------+-------------------------------------------------+-------------------------+
| 16.994521521610128|                               0.2602294959362543|               10-20 mins|
| 16.498508623452825|                               0.2152663797451226|                5-10 mins|
| 20.461489266812094|                               0.3074613545115925|               20-30 mins|
|-14.435587253389443|                               0.3066885786108011|     negative time - I...|
|  24.05022920905049|                               0.3805705749984456|         at least 30 mins|
|  38.65416235197026|                              0.17016166282864423|                 0-5 mins|
+-------------------+-------------------------------------------------+-------------------------+



f. Which duration bin will you advise a taxi driver to target to maximise his income?
The lower the band generally correlates with the higher the income per km (or lower km per $). However, this does not take into account the wait times of 